In [1]:
# import core packages
import warnings
warnings.filterwarnings("ignore")
from itertools import combinations
import os

# import semi-core packages
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline
plt.style.use('seaborn-v0_8-poster')
import numpy as np
import pandas as pd
from multiprocessing import Pool

# import open2c libraries
import bioframe

import cooler
import cooltools

from packaging import version
if version.parse(cooltools.__version__) < version.parse('0.5.2'):
    raise AssertionError("tutorial relies on cooltools version 0.5.2 or higher,"+
                         "please check your cooltools version and update to the latest")

# count cpus
num_cpus = os.getenv('SLURM_CPUS_PER_TASK')
if not num_cpus:
    num_cpus = os.cpu_count()
num_cpus = int(num_cpus)

In [2]:
cooltools.__version__

'0.6.1'

In [ ]:
resolution = 1000
clr1 = cooler.Cooler('cooler/1.ice.cool')
clr2 = cooler.Cooler('cooler/2.ice.cool')
clr3 = cooler.Cooler('cooler/3.ice.cool')
clr4 = cooler.Cooler('cooler/4.ice.cool')

In [ ]:
arms = pd.read_table('')
arms

In [ ]:
# cvd == contacts-vs-distance
cvd1 = cooltools.expected_cis(
    clr=clr1,
    view_df=arms,
    smooth=False,
    aggregate_smoothed=False,
    nproc=1 #if you do not have multiple cores available, set to 1
)

# cvd == contacts-vs-distance
cvd2 = cooltools.expected_cis(
    clr=clr2,
    view_df=arms,
    smooth=False,
    aggregate_smoothed=False,
    nproc=1 #if you do not have multiple cores available, set to 1
)

# cvd == contacts-vs-distance
cvd3 = cooltools.expected_cis(
    clr=clr3,
    view_df=arms,
    smooth=False,
    aggregate_smoothed=False,
    nproc=1 #if you do not have multiple cores available, set to 1
)

# cvd == contacts-vs-distance
cvd4 = cooltools.expected_cis(
    clr=clr4,
    view_df=arms,
    smooth=False,
    aggregate_smoothed=False,
    nproc=1 #if you do not have multiple cores available, set to 1
)

In [ ]:
cvd_smooth_agg1 = cooltools.expected_cis(
    clr=clr1,
    view_df=arms,
    smooth=True,
    aggregate_smoothed=True,
    smooth_sigma=0.1,
    nproc=num_cpus
)

cvd_smooth_agg2 = cooltools.expected_cis(
    clr=clr2,
    view_df=arms,
    smooth=True,
    aggregate_smoothed=True,
    smooth_sigma=0.1,
    nproc=num_cpus
)

cvd_smooth_agg3 = cooltools.expected_cis(
    clr=clr3,
    view_df=arms,
    smooth=True,
    aggregate_smoothed=True,
    smooth_sigma=0.1,
    nproc=num_cpus
)

cvd_smooth_agg4 = cooltools.expected_cis(
    clr=clr4,
    view_df=arms,
    smooth=True,
    aggregate_smoothed=True,
    smooth_sigma=0.1,
    nproc=num_cpus
)

In [ ]:
cvd_smooth_agg1['s_bp'] = cvd_smooth_agg1['dist']* resolution
cvd_smooth_agg1['balanced.avg.smoothed.agg'].loc[cvd_smooth_agg1['dist'] < 2] = np.nan

cvd_smooth_agg2['s_bp'] = cvd_smooth_agg2['dist']* resolution
cvd_smooth_agg2['balanced.avg.smoothed.agg'].loc[cvd_smooth_agg2['dist'] < 2] = np.nan

cvd_smooth_agg3['s_bp'] = cvd_smooth_agg3['dist']* resolution
cvd_smooth_agg3['balanced.avg.smoothed.agg'].loc[cvd_smooth_agg3['dist'] < 2] = np.nan

cvd_smooth_agg4['s_bp'] = cvd_smooth_agg4['dist']* resolution
cvd_smooth_agg4['balanced.avg.smoothed.agg'].loc[cvd_smooth_agg4['dist'] < 2] = np.nan


f, ax = plt.subplots(1,1)

for region in arms['name']:
    ax.loglog(
        cvd_smooth_agg1['s_bp'].loc[cvd_smooth_agg1['region1']==region],
        cvd_smooth_agg1['balanced.avg.smoothed.agg'].loc[cvd_smooth_agg1['region1']==region],
  
        cvd_smooth_agg2['s_bp'].loc[cvd_smooth_agg2['region1']==region],
        cvd_smooth_agg2['balanced.avg.smoothed.agg'].loc[cvd_smooth_agg2['region1']==region],
        
        cvd_smooth_agg3['s_bp'].loc[cvd_smooth_agg3['region1']==region],
        cvd_smooth_agg3['balanced.avg.smoothed.agg'].loc[cvd_smooth_agg3['region1']==region],
        
        cvd_smooth_agg4['s_bp'].loc[cvd_smooth_agg4['region1']==region],
        cvd_smooth_agg4['balanced.avg.smoothed.agg'].loc[cvd_smooth_agg4['region1']==region],
    )
    ax.set(
        xlabel='separation, bp',
        ylabel='IC contact frequency')
    ax.set_aspect(1.0)
    ax.grid(lw=0.5)
    

In [ ]:
# Just take a single value for each genomic separation
cvd_merged1 = cvd_smooth_agg1.drop_duplicates(subset=['dist'])[['s_bp', 'balanced.avg.smoothed.agg']]

cvd_merged2 = cvd_smooth_agg2.drop_duplicates(subset=['dist'])[['s_bp', 'balanced.avg.smoothed.agg']]

cvd_merged3 = cvd_smooth_agg3.drop_duplicates(subset=['dist'])[['s_bp', 'balanced.avg.smoothed.agg']]

cvd_merged4 = cvd_smooth_agg4.drop_duplicates(subset=['dist'])[['s_bp', 'balanced.avg.smoothed.agg']]

In [ ]:
# Calculate derivative in log-log space
der1 = np.gradient(np.log(cvd_merged1['balanced.avg.smoothed.agg']),
                  np.log(cvd_merged1['s_bp']))

der2 = np.gradient(np.log(cvd_merged2['balanced.avg.smoothed.agg']),
                  np.log(cvd_merged2['s_bp']))

der3 = np.gradient(np.log(cvd_merged3['balanced.avg.smoothed.agg']),
                  np.log(cvd_merged3['s_bp']))

der4 = np.gradient(np.log(cvd_merged4['balanced.avg.smoothed.agg']),
                  np.log(cvd_merged4['s_bp']))

In [ ]:
f, axs = plt.subplots(
    figsize=(6.5,13),
    nrows=2,
    gridspec_kw={'height_ratios':[6,2]},
    sharex=True)
ax = axs[0]
ax.loglog(
    cvd_merged1['s_bp'],
    cvd_merged1['balanced.avg.smoothed.agg'],
    #'-'
    cvd_merged2['s_bp'],
    cvd_merged2['balanced.avg.smoothed.agg'],
    
    cvd_merged3['s_bp'],
    cvd_merged3['balanced.avg.smoothed.agg'],
    
    cvd_merged4['s_bp'],
    cvd_merged4['balanced.avg.smoothed.agg'],
    '-'
)

ax.set(
    ylabel='IC contact frequency',
    xlim=(1e3,1e8)
)
ax.set_aspect(1.0)
ax.grid(lw=0.5)


ax = axs[1]
ax.semilogx(
    cvd_merged1['s_bp'],
    der1,
    #alpha=0.5
    
    cvd_merged2['s_bp'],
    der2,
    #alpha=0.5
    
    cvd_merged3['s_bp'],
    der3,
    #alpha=0.5
    
    cvd_merged4['s_bp'],
    der4,
    alpha=0.5
)

ax.set(
    xlabel='separation, bp',
    ylabel='slope')

ax.grid(lw=0.5)



# adjust x axis scale
ax.legend(['1', '2', '3', '4'], loc="center left", bbox_to_anchor=(1, 0.5))

# save 
plt.savefig('1kb_Ps_smooth_curve.pdf', bbox_inches='tight')